welcome to my notebook, here i will guide you through my first attempt of constructing a simple CNN classifier for the MNIST dataset. First, we load the data, as we can see the training set consists of 42000 samples and the test set consists of 28000 samples.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl

data=pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
datates=pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
print(data.shape)
print(datates.shape)
print(data)


My next step is to split the data into several smaller dataset as 42000 samples is quite a lot and will be time consuming to process all of it. Here i used 40% of the original training set for training the algorithm(datrainx and y) and another smaller batch of 1000 (modtestx and y) for testing several NN architectures. This will still take quite a while to train but i still find it acceptable.

In [ ]:
#choose percentage of data to be used for training
random=np.random.rand(42000)<0.9#<<by changing this
#training data 40%
datrainx=data.loc[random]
datrainy=data.loc[random,'label']
datrainx.drop('label',axis=1,inplace=True)
datrainx.reset_index(drop=True,inplace=True)
datrainy=datrainy.reset_index(drop=True)

#validation data 15% of the data
daval=data.loc[~random]
r=np.random.rand(daval.shape[0])<0.15
davalx=daval.loc[r]
davalx.drop('label',axis=1,inplace=True)
davaly=daval.loc[r,'label']
davalx.reset_index(drop=True,inplace=True)
davaly=davaly.reset_index(drop=True)
#data for model selection
#model selection data 20% of validation
rt=np.random.rand(davalx.shape[0])<0.20
modtestx=davalx.loc[rt]
modtesty=davaly.loc[rt]
modtestx.reset_index(drop=True,inplace=True)
modtesty=modtesty.reset_index(drop=True)
print("Training Data:")
print(datrainx.shape)
print(datrainy.shape)
print("Validation Data:")
print(davalx.shape)
print(davaly.shape)
print("Model Selection Data:")
print(modtestx.shape)
print(modtesty.shape)

Next, i created a function to reshape the data to feed the algorithm, 28,28 per sample for the mlp and 28,28,1 for the CNN. We also need to scale the data for better performance (you could try using the original data and compare the performance vs the scaled one) by dividing it with 255 (minmax scaling).

In [ ]:
def shapeshift(a):
    return np.array(a).reshape(a.shape[0],28,28)
def shapeshiftconv(a):
    return np.array(a).reshape(a.shape[0],28,28,1)
#change the shape of the data to reflect the image's characteristic, conv for those that will be fed to
#CNNs (28,28,1)
datatest=shapeshiftconv(datates)
datrainx=shapeshift(datrainx)
davalx=shapeshift(davalx)
modtestx=shapeshift(modtestx)
datrainxconv=shapeshiftconv(datrainx)
davalxconv=shapeshiftconv(davalx)
modtestxconv=shapeshiftconv(modtestx)
print("Model Selection Data:")
print(modtestx.shape)
print(modtestxconv.shape)
print("Training Data:")
print(datrainxconv.shape)
print("Validation Data:")
print(davalxconv.shape)
print("Label of Image:")
plt.imshow(datrainx[3],cmap=mpl.cm.binary)
print(datrainy[3])
plt.show()
datatest=datatest/255
datrainx=datrainx/255
davalx=davalx/255
modtestx=modtestx/255
modtestxconv=modtestxconv/255
datrainxconv=datrainxconv/255
print(davalxconv.shape)

Next is the construction of the NN. I tried to keep the number of layers/ parameter as low as possible (or at least until a level that i found acceptable) while minding the accuracy. Some simpler NN could perform better than the deeper/more complex ones as they tend to generalize better. I tested three models in this section: a MLP, CNN with alternating convolutional and pooling layer, and a CNN with double convolutional layers as the first layers. I tried it several times using different parameters and number of neurons, during which some more complex models with more neurons performed worse than simpler ones.

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split


model=keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(200,activation='relu'))
model.add(keras.layers.Dense(200,activation='relu'))
model.add(keras.layers.Dense(100,activation='relu'))
model.add(keras.layers.Dense(10,activation='softmax'))
model.summary()

model1=keras.models.Sequential()
model1.add(keras.layers.Conv2D(64,(4,4),activation='relu',input_shape=[28,28,1],data_format='channels_last'))
model1.add(keras.layers.MaxPool2D((2,2)))
model1.add(keras.layers.Conv2D(128,(4,4),activation='relu'))
model1.add(keras.layers.MaxPool2D((2,2)))
model1.add(keras.layers.Flatten())
model1.add(keras.layers.Dense(100,activation='relu'))
model1.add(keras.layers.Dense(10,activation='softmax'))
model1.summary()

model2=keras.models.Sequential()
model2.add(keras.layers.Conv2D(32,(4,4),activation='relu',input_shape=[28,28,1],data_format='channels_last'))
model2.add(keras.layers.Conv2D(64,(4,4),activation='relu'))
model2.add(keras.layers.MaxPool2D((2,2)))
model2.add(keras.layers.Conv2D(48,(4,4),activation='relu'))
model2.add(keras.layers.MaxPool2D((2,2)))
model2.add(keras.layers.Flatten())
model2.add(keras.layers.Dense(200,activation='relu'))
model2.add(keras.layers.Dense(10,activation='softmax'))
model2.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
model1.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
model2.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
hisnn=model.fit(modtestx[:700],modtesty[:700],epochs=30,validation_data=(modtestx[701:],modtesty[701:]))
hisconv=model1.fit(modtestxconv[:700],modtesty[:700],epochs=30,validation_data=(modtestxconv[701:],modtesty[701:]))
hisconv2=model2.fit(modtestxconv[:700],modtesty[:700],epochs=30,validation_data=(modtestxconv[701:],modtesty[701:]))
x=range(30)
plt.plot(x,hisnn.history['loss'],color='r',label='mlp')
plt.plot(x,hisconv.history['loss'],color='b',label='Conv')
plt.plot(x,hisconv2.history['loss'],color='g',label='Doubleconv')
plt.legend()
plt.show()

As we can see, the model with double convolutional layers outperforms the other two in terms of accuracy although it took far longer to train than the other 2. I chose the double convolutional CNN as our model. The last step is to train the model on the training data and then predict the result.

In [ ]:
#reinitializing the model
model2.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
model2.fit(datrainxconv,datrainy,epochs=30,validation_data=(davalxconv,davaly))

result=pd.DataFrame({'ImageId':range(1,datates.shape[0]+1)})
result['Label']=model2.predict_classes(datatest)
result.set_index('ImageId',inplace=True)
print(result)
result.to_csv('result.csv')